In [6]:
!pip install mysql-connector

     ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
     --------------------------------------- 0.1/11.9 MB 655.4 kB/s eta 0:00:18
     --------------------------------------- 0.1/11.9 MB 599.1 kB/s eta 0:00:20
      -------------------------------------- 0.2/11.9 MB 784.3 kB/s eta 0:00:15
      -------------------------------------- 0.2/11.9 MB 784.3 kB/s eta 0:00:15
      -------------------------------------- 0.2/11.9 MB 784.3 kB/s eta 0:00:15
      -------------------------------------- 0.2/11.9 MB 528.4 kB/s eta 0:00:23
      -------------------------------------- 0.2/11.9 MB 528.4 kB/s eta 0:00:23
      -------------------------------------- 0.2/11.9 MB 528.4 kB/s eta 0:00:23
      -------------------------------------- 0.2/11.9 MB 528.4 kB/s eta 0:00:23
      -------------------------------------- 0.2/11.9 MB 528.4 k


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import mysql.connector
import os
import MySQLdb
# List of CSV files and their corresponding table names
csv_files = [
    ('customers.csv', 'customers'),
    ('orders.csv', 'orders'),
    ('sellers.csv', 'sellers'),
    ('products.csv', 'products'),
    ('payments.csv', 'Payment'),
    ('geolocation.csv', 'geolocation'),
    ('order_items.csv', 'order_items')
        # Added payments.csv for specific handling
]

# Connect to the MySQL database
host = 'localhost'
user = 'root'
password = 'mohid708@'
database = 'ecommerce'
conn = MySQLdb.connect(host=host, user=user, passwd=password, db=database)

cursor = conn.cursor()

# Folder containing the CSV files
folder_path = 'C:/Users/lenovo/Desktop/SQL+Python(Project)/Datasets'

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Replace NaN with None to handle SQL NULL
    df = df.where(pd.notnull(df), None)
    
    # Debugging: Check for NaN values
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    # Insert DataFrame data into the MySQL table
    for _, row in df.iterrows():
        # Convert row to tuple and handle NaN/None explicitly
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)

    # Commit the transaction for the current CSV file
    conn.commit()

# Close the connection

C:\Users\lenovo\AppData\Local\Temp\ipykernel_2696\2181929779.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Processing customers.csv
NaN values before replacement:
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

Processing orders.csv
NaN values before replacement:
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

Processing sellers.csv
NaN values before replacement:
seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64

Processing products.csv
NaN values before replacement:
product_id                      0
product category              610
product_name_length           610
product_description_length    610
product_photos_qty            610
prod

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Error connecting to MySQL: (1146, "Table 'ecommerce.customers' doesn't exist")
